# Parameters

In [1]:
dest_dir = "/tmp/ghe_20210701"

# Read data and reference tables

In [2]:
from owid import catalog
import json

In [4]:
from etl.paths import DATA_DIR, BASE_DIR as base_path

In [5]:
countries = catalog.Dataset(DATA_DIR / "garden/regions/2023-01-01/regions")["regions"]

In [6]:
countries.head()

,name,short_name,region_type,is_historical,defined_by,members,aliases,related,cow_code,cow_letter,...,iso_alpha2,iso_alpha3,kansas_code,legacy_country_id,legacy_entity_id,marc_code,ncd_code,penn_code,unctad_code,wikidata_code
code,,,,,,,,,,,,,,,,,,,,,
ABW,Aruba,Aruba,country,False,owid,NaN,NaN,NaN,<NA>,NaN,...,AW,ABW,ABW,561,219,AW,NaN,ABW,ARU,Q21203
AFG,Afghanistan,Afghanistan,country,False,owid,NaN,NaN,NaN,700,AFG,...,AF,AFG,AFG,562,15,AF,AFGN,AFG,AFG,Q889
AGO,Angola,Angola,country,False,owid,NaN,NaN,NaN,540,ANG,...,AO,AGO,AGO,563,19,AO,ANGL,AGO,ANG,Q916
AIA,Anguilla,Anguilla,country,False,owid,NaN,NaN,NaN,<NA>,NaN,...,AI,AIA,AIA,564,228,AM,NaN,AIA,ANL,Q25228
ALA,Aland Islands,Aland Islands,country,False,owid,NaN,"[""\u00c3\u0081EEland Islands"", ""\u00c5land Isl...",NaN,<NA>,NaN,...,AX,ALA,NaN,791,61373,NaN,NaN,NaN,NaN,Q5689


In [7]:
ghe_meadow = catalog.Dataset((base_path / "data/meadow/who/2021-07-01/ghe").as_posix())

In [8]:
ghe_estimates = ghe_meadow["estimates"]

In [9]:
ghe_estimates.shape

(7191900, 9)

In [10]:
ghe_estimates.head()

population  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders BTSX     YEARS45-49           6285   
                  Syphilis              MLE      YEARS1-4             3729   
                  Cataracts             FMLE     YEARS15-19           3736   
                  Peptic ulcer disease  BTSX     ALLAges            112002   
                  Drug use disorders    MLE      YEARS85PLUS           366   

                                                                deaths  \
country_code year ghe_cause_title       sex_code agegroup_code           
GRD          2019 Alcohol use disorders BTSX     YEARS45-49        0.5   
                  Syphilis              MLE      YEARS1-4          0.0   
                  Cataracts             FMLE     YEARS15-19        0.0   
                  Peptic ulcer disease  BTSX     ALLAges           4.7   
                  Drug use disorders    MLE      YEARS85PLUS       0.0   

                                                                 deaths_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders BTSX     YEARS45-49     8.200000e-05   
                  Syphilis              MLE      YEARS1-4       1.000000e-08   
                  Cataracts             FMLE     YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  BTSX     ALLAges        4.190000e-05   
                  Drug use disorders    MLE      YEARS85PLUS    8.524000e-05   

                                                                deaths_100k  \
country_code year ghe_cause_title       sex_code agegroup_code                
GRD          2019 Alcohol use disorders BTSX     YEARS45-49             8.2   
                  Syphilis              MLE      YEARS1-4               0.0   
                  Cataracts             FMLE     YEARS15-19             0.0   
                  Peptic ulcer disease  BTSX     ALLAges                4.2   
                  Drug use disorders    MLE      YEARS85PLUS            8.5   

                                                                      daly  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders BTSX     YEARS45-49      42.099998   
                  Syphilis              MLE      YEARS1-4         0.000000   
                  Cataracts             FMLE     YEARS15-19       0.000000   
                  Peptic ulcer disease  BTSX     ALLAges        126.099998   
                  Drug use disorders    MLE      YEARS85PLUS      0.400000   

                                                                   daly_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders BTSX     YEARS45-49     6.695200e-03   
                  Syphilis              MLE      YEARS1-4       5.900000e-07   
                  Cataracts             FMLE     YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  BTSX     ALLAges        1.126140e-03   
                  Drug use disorders    MLE      YEARS85PLUS    1.031580e-03   

                                                                 daly_100k  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders BTSX     YEARS45-49     669.500000   
                  Syphilis              MLE      YEARS1-4         0.100000   
                  Cataracts             FMLE     YEARS15-19       0.000000   
                  Peptic ulcer disease  BTSX     ALLAges        112.599998   
                  Drug use disorders    MLE      YEARS85PLUS    103.199997   

                                                                causegroup  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders BTSX     YEARS45-49             

In [11]:
country_codes = set(ghe_estimates.index.unique("country_code"))

In [12]:
len(country_codes)

183

In [13]:
owid_country_codes = set(countries.index)

In [14]:
len(owid_country_codes)

285

In [15]:
country_codes - owid_country_codes

set()

💡 The set of (country codes in GHE) minus (owid table country codes) is empty, so we don't need to do any further harmonization here.

In [16]:
ghe_estimates.index.unique("sex_code")

CategoricalIndex(['BTSX', 'MLE', 'FMLE'], categories=['BTSX', 'FMLE', 'MLE'], ordered=False, dtype='category', name='sex_code')

In [17]:
with open(base_path / "etl" / "steps" / "data" / "garden" / "who" / "2021-07-01" / "ghe.mapping.sex.json") as f:
    ghe_to_owid_sex_codes = json.load(f)

In [18]:
ghe_to_owid_sex_codes

{'BTSX': 'both', 'MLE': 'male', 'FMLE': 'female'}

In [19]:
index_names = ghe_estimates.index.names

In [20]:
ghe_estimates.reset_index(inplace=True)

In [21]:
ghe_estimates["sex_code"] = ghe_estimates["sex_code"].cat.rename_categories(ghe_to_owid_sex_codes)

In [22]:
ghe_estimates.set_index(index_names, inplace=True)

In [23]:
ghe_estimates.head()

population  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders both     YEARS45-49           6285   
                  Syphilis              male     YEARS1-4             3729   
                  Cataracts             female   YEARS15-19           3736   
                  Peptic ulcer disease  both     ALLAges            112002   
                  Drug use disorders    male     YEARS85PLUS           366   

                                                                deaths  \
country_code year ghe_cause_title       sex_code agegroup_code           
GRD          2019 Alcohol use disorders both     YEARS45-49        0.5   
                  Syphilis              male     YEARS1-4          0.0   
                  Cataracts             female   YEARS15-19        0.0   
                  Peptic ulcer disease  both     ALLAges           4.7   
                  Drug use disorders    male     YEARS85PLUS       0.0   

                                                                 deaths_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders both     YEARS45-49     8.200000e-05   
                  Syphilis              male     YEARS1-4       1.000000e-08   
                  Cataracts             female   YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  both     ALLAges        4.190000e-05   
                  Drug use disorders    male     YEARS85PLUS    8.524000e-05   

                                                                deaths_100k  \
country_code year ghe_cause_title       sex_code agegroup_code                
GRD          2019 Alcohol use disorders both     YEARS45-49             8.2   
                  Syphilis              male     YEARS1-4               0.0   
                  Cataracts             female   YEARS15-19             0.0   
                  Peptic ulcer disease  both     ALLAges                4.2   
                  Drug use disorders    male     YEARS85PLUS            8.5   

                                                                      daly  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders both     YEARS45-49      42.099998   
                  Syphilis              male     YEARS1-4         0.000000   
                  Cataracts             female   YEARS15-19       0.000000   
                  Peptic ulcer disease  both     ALLAges        126.099998   
                  Drug use disorders    male     YEARS85PLUS      0.400000   

                                                                   daly_rate  \
country_code year ghe_cause_title       sex_code agegroup_code                 
GRD          2019 Alcohol use disorders both     YEARS45-49     6.695200e-03   
                  Syphilis              male     YEARS1-4       5.900000e-07   
                  Cataracts             female   YEARS15-19     0.000000e+00   
                  Peptic ulcer disease  both     ALLAges        1.126140e-03   
                  Drug use disorders    male     YEARS85PLUS    1.031580e-03   

                                                                 daly_100k  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders both     YEARS45-49     669.500000   
                  Syphilis              male     YEARS1-4         0.100000   
                  Cataracts             female   YEARS15-19       0.000000   
                  Peptic ulcer disease  both     ALLAges        112.599998   
                  Drug use disorders    male     YEARS85PLUS    103.199997   

                                                                causegroup  \
country_code year ghe_cause_title       sex_code agegroup_code               
GRD          2019 Alcohol use disorders both     YEARS45-49             

In [24]:
ghe_garden = catalog.Dataset.create_empty(dest_dir)

In [25]:
ghe_garden.metadata = ghe_meadow.metadata
ghe_garden.save()

In [26]:
ghe_garden.add(ghe_estimates)

In [ ]:
ghe_garden.save()